# Lab 06 : Image Data Modeling


#### Lab Overview

This workshop focuses on building a machine learning model that is able to classify images using neural networks.


#### Dataset Loading

**Definition:**
Organize your images into class-named subfolders and use TensorFlow’s high-level API to load, resize, batch, and normalize them for training.

**Directory Structure:**

```
data/
├── train/
│   ├── class_a/
│   │   ├── img001.jpg
│   │   └── img002.jpg
│   └── class_b/
│       ├── img101.jpg
│       └── img102.jpg
├── validation/
│   ├── class_a/
│   └── class_b/
└── test/
    ├── class_a/
    └── class_b/
```

**Loading the images from directory**

```python
import tensorflow as tf

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'directory path',
    labels=,
    label_mode=,
    image_size=,
    batch_size=,
    shuffle=,
    seed=42
)

```

**Explanation of Key Parameters:**

- **`directory`**: Root path containing class subfolders. ex: `data/train` in case of loading training dataset
- **`labels='inferred'`**: Automatically assign labels based on subfolder names.
- **`label_mode`**:

  - `'categorical'` → one-hot vectors,
  - `'int'` → integer indices,
  - `'binary'` → single 0/1 label (for 2-class problems).

- **`image_size=(H, W)`**: Resize each image to this shape. (based on model and image resolution)
- **`batch_size`**: How many images to return per batch (affects training speed/memory).
- **`shuffle` & `seed`**: Randomize sample order; seed ensures reproducibility. Usually shuffle is `True` for training and `False` for testing and validation.


#### Data Augmentation

**Definition:**
Dynamically apply random transformations to your training images to increase dataset diversity and help your model generalize.

```python
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1. Configure augmentation parameters
gen = ImageDataGenerator(
    rotation_range=30,            # Random rotations in the range ±30°
    width_shift_range=0.1,        # Horizontal shifts up to 10% of image width
    height_shift_range=0.1,       # Vertical shifts up to 10% of image height
    brightness_range=[0.8, 1.2],  # Random brightness adjustments between 80% and 120%
    horizontal_flip=True,         # Randomly flip images left ↔ right
    zoom_range=0.2,               # Random zoom in range [80%, 120%]
    shear_range=0.1               # Shear intensity (in radians) up to ±0.1
)

# 2. Build an iterator that reads from directory and applies augmentations
iterator = gen.flow_from_directory(
    'data/train',                 # Root directory containing subfolders per class
    target_size=(224, 224),       # Resize all images to 224×224
    batch_size=32,                # Number of images per batch
    class_mode='categorical'      # Return one-hot encoded labels (use 'binary' for 2 classes)
)
```

**Explanation of Key Parameters:**

- `rotation_range`: Degree range for random rotations.
- `width_shift_range` & `height_shift_range`: Fraction of total width/height for random translations.
- `brightness_range`: Tuple `[min, max]` to scale pixel intensity.
- `horizontal_flip`: Boolean to enable random left–right flips.
- `zoom_range`: Float or `[min, max]` for random zoom.
- `shear_range`: Shear angle in radians for affine transformations.
- `flow_from_directory`:

  - **`'data/train'`**: folder with subdirectories named by class.
  - **`target_size`**: uniformly resizes input images.
  - **`batch_size`**: number of samples per generated batch.
  - **`class_mode`**: how labels are returned (`'categorical'`, `'binary'`, `'sparse'`, or `None`).


#### CNN Architecture in Keras

to get layers to build the architectural blocks of the CNN you need the following import:

`from tensorflow.keras import layers`

Break your model into two logical parts:

---

##### 1. Feature Extraction Block

**Definition:** Stacks of convolutional filters + downsampling to learn spatial hierarchies of features (edges → textures → shapes).

| Layer                  | Purpose & When to Use                                                           | Syntax Example                                                                         |
| ---------------------- | ------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------- |
| **Conv2D**             | Learn local patterns; start with small filters (3×3) and double filters deeper. | `layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(224,224,3))` |
| **BatchNormalization** | Stabilize and accelerate training by normalizing activations.                   | `layers.BatchNormalization()`                                                          |
| **Activation**         | Apply non-linearity (if not in Conv2D).                                         | `layers.Activation('relu')`                                                            |
| **MaxPooling2D**       | Downsample spatial dimensions by taking max in each region.                     | `layers.MaxPooling2D(pool_size=(2,2), strides=(2,2))`                                  |
| **Dropout**            | Regularize; randomly zero inputs to reduce overfitting.                         | `layers.Dropout(0.25)`                                                                 |

**Typical order in one block:**

```python
layers.Conv2D(...)
layers.BatchNormalization()
layers.Activation('relu')        # or use activation in Conv2D
layers.MaxPooling2D(...)
layers.Dropout(0.25)
```

---

##### 2. Classification Block

**Definition:** Map flattened feature maps to class probabilities via dense layers.

| Layer                               | Purpose & When to Use                                              | Syntax Example                                    |
| ----------------------------------- | ------------------------------------------------------------------ | ------------------------------------------------- |
| **Flatten**                         | Collapse spatial dims to 1D vector                                 | `layers.Flatten()`                                |
| **GlobalAveragePooling2D** _(alt.)_ | Reduce each feature map to its average; fewer params than Flatten. | `layers.GlobalAveragePooling2D()`                 |
| **Dense**                           | Fully-connected layer to learn combinations of features.           | `layers.Dense(128, activation='relu')`            |
| **Dropout**                         | Further regularization before final output.                        | `layers.Dropout(0.5)`                             |
| **Output Dense (multi-class)**      | Final logits → probabilities across N classes.                     | `layers.Dense(num_classes, activation='softmax')` |
| **Output Dense (binary)**           | Single probability for two classes.                                | `layers.Dense(1, activation='sigmoid')`           |

**Typical order:**

```python
layers.Flatten()                     # or GlobalAveragePooling2D()
layers.Dense(128, activation='relu')
layers.Dropout(0.5)
layers.Dense(num_classes, activation='softmax')
```

To connect layers you put the output of the previous layer as the input of the current layer, example:

```python
input = keras.Input(shape=input_shape)
model = layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(224,224,3))(input)
model = layers.Flatten()(model)
```

To get the whole model's architecture use the following: `model.summary()`


#### Model Training

**Definition:**
Compile your CNN by specifying the optimizer, loss, and metrics, then train it on your datasets using `model.fit()` without any callbacks.

```python
# 1. Compile the model
model.compile(
    optimizer='adam',                  # e.g. 'adam', 'sgd', or a configured tf.keras.optimizers.Optimizer
    loss='categorical_crossentropy',   # for one-hot multi-class; use 'binary_crossentropy' for 2-class
    metrics=['accuracy']               # list of metrics to track during training
)

# 2. Train the model
history = model.fit(
    train_ds,                          # tf.data.Dataset (or NumPy arrays) for training
    validation_data=val_ds,           # tf.data.Dataset (or tuple) for validation
    epochs=30                          # number of full passes through the training data
)
```

**Explanation of Key Arguments:**

- **`optimizer`**: algorithm that updates network weights (e.g., Adam adapts learning rates per parameter).
- **`loss`**: objective function to minimize; chosen based on label encoding (categorical vs. binary).
- **`metrics`**: additional performance measures displayed each epoch.
- **`train_ds`**: your preprocessed and batched training dataset.
- **`validation_data`**: held-out dataset to track generalization at the end of each epoch.
- **`epochs`**: how many times the model sees the entire training set.

The returned `history` object contains `history.history['loss']`, `['accuracy']`, `['val_loss']`, and `['val_accuracy']`, which you can plot to visualize training dynamics.


#### Model Evaluation

**Definition:**
Assess the trained model’s performance on held-out test data by computing loss and metrics, and (optionally) generating detailed reports like confusion matrices.

```python
# 1. Compute loss & accuracy on the test set
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
```

```python
# 2. (Optional) Detailed classification metrics
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Gather true labels and predictions
y_true = np.concatenate([y.numpy() for x, y in test_ds], axis=0)
y_pred_probs = model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true_idx = np.argmax(y_true, axis=1)

# Print confusion matrix and classification report
print("Confusion Matrix:")
print(confusion_matrix(y_true_idx, y_pred))
print("\nClassification Report:")
print(classification_report(y_true_idx, y_pred, target_names=test_ds.class_names))
```

**Explanation of Key Steps:**

- **`model.evaluate(test_ds)`**:

  - Runs the model on each batch in `test_ds`.
  - Returns the loss and any metrics (e.g., accuracy) defined at compile time.

- **`model.predict(test_ds)`**:

  - Generates raw predictions (probabilities) for each test sample.

- **`np.argmax(...)`**:

  - Converts one-hot or probability vectors to class indices.

- **`classification_report` / `confusion_matrix`**:

  - From scikit-learn, provides precision/recall/F1 for each class and a matrix of true vs. predicted labels.


#### Hands-on Activity

For the assigned dataset perform the following tasks:<br>
**Task 1** : Structure your directory as shown in the Dataset Loading section<br>
**Task 2** : Load the dataset and split it into training, validation, and test sets<br>
**Task 3** : Create a new directory containing four subdirectories—one for each class<br>
**Task 4** : Build the model architecture (feature-extraction and classification blocks), explaining your design choices as needed<br>
**Task 5** : Plot the training curves (e.g., loss and accuracy) and assess whether the model is overfitting or underfitting<br>
**Task 6** : Evaluate the final model on the test set and interpret the results<br>
**Task 7** : Choose a preprocessing technique from the previous lab, apply it to your images, then repeat Tasks 1–6 on the preprocessed dataset. Discuss whether performance improved or declined—and why.<br>
